In [1]:
import pywikibot as pwb

In [18]:
frame = pd.read_csv("./sampling-frame.tsv", sep = "\t")
frame.head()

,user,home_wiki,global_edits,enwiki_edits,active_months
0,! Bikkit !,dewiki,559.2,1.0,3
1,!NewLondon31,jawiki,46.0,1.0,2
2,!Silent,ptwiki,22349.8,2.0,3
3,"""Colorado Campeão""!",ptwiki,422.0,4.0,3
4,"""quasi"" tuttologo",itwiki,115.0,1.0,3


In [3]:
strata = pd.read_table("strata_definitions.tsv")
strata.head()

,code,audience,home_wiki,also_active_enwiki,min_sample,sample,population
0,01-AEAR,AE,arwiki,0,150,225,NaN
1,02-AEZH,AE,zhwiki,0,150,225,NaN
2,03-AEFR,AE,frwiki,0,150,225,NaN
3,04-AEES,AE,eswiki,0,375,563,NaN
4,05-AERU,AE,ruwiki,0,150,225,NaN


In [4]:
# To-do: what about new Wikipedias since last year?
wps_by_group = pd.read_table("wikipedias_by_group.tsv")

# Remove human-readable language name column
wps_by_group = wps_by_group[["site_key", "group"]]

wps_by_group.head()

,site_key,group
0,aawiki,ssa
1,abwiki,mena
2,acewiki,asia
3,adywiki,cee
4,afwiki,ssa


In [19]:
frame = frame.merge(wps_by_group, how = "left", left_on = "home_wiki", right_on = "site_key" )
frame = frame.drop("site_key", axis = 1)
frame.head()

,user,home_wiki,global_edits,enwiki_edits,active_months,group
0,! Bikkit !,dewiki,559.2,1.0,3,dewiki
1,!NewLondon31,jawiki,46.0,1.0,2,jawiki
2,!Silent,ptwiki,22349.8,2.0,3,ptwiki
3,"""Colorado Campeão""!",ptwiki,422.0,4.0,3,ptwiki
4,"""quasi"" tuttologo",itwiki,115.0,1.0,3,itwiki


In [23]:
frame["group"].count()

47512

In [27]:
frame["group"] = frame["group"].fillna("other")
frame["group"].count()

52496

In [29]:
frame.groupby("group")["user"].count()

group
arwiki      415
asia       1795
cee        3483
dewiki     3815
enwiki    18126
eswiki     2456
frwiki     3097
itwiki     1730
jawiki     3198
mena       1621
nlwiki      734
other      5006
ptwiki      964
ruwiki     2340
ssa          56
weur       1781
zhwiki     1879
Name: user, dtype: int64

In [ ]:
queries = []

for stratum in strata.itertuples(index = False):
    if stratum.audience == "AE":
        query = "global_edits < 600"
    else:
        query = "global_edits >= 600"
    
    query += (" & home_wiki.isin({})".format(list(wiki_groups[stratum.home_wiki])))
    
    if stratum.home_wiki != "enwiki":
        if stratum.also_active_enwiki:
            query += " & enwiki >= 30"
        else:
            query += " & (enwiki < 30 | enwiki.isnull())"
    
    queries.append(query)
    
strata["query"] = queries

for query in strata["query"][:5]:
    print(query)

In [ ]:
def sample_stratum(stratum_query, sample_size):
    """
    returns a tuple consisting of (1) the population count for that subgroup and
    (2) a list of the sampled users in MassMessage format 
    """ 
    stratum = frame.query(stratum_query)
    stratum_size = len(stratum)
    
    if sample_size == 0:
        sample = None
    elif sample_size > stratum_size:
        sample = stratum[["user_name", "home_wiki"]]
    else:
        sample = stratum.sample(n = sample_size)[["user_name", "home_wiki"]]
        
    if sample is None:
        sample_list = None
    else:
        sample_list = [
            i for i in sample.itertuples(index = False, name = "user")
        ]

    return (stratum_size, sample_list)

In [ ]:
results = [[], []]

for row in strata.itertuples(index = False):
    size, sample = sample_stratum(row.query, row.sample)
    results[0].append(size)
    results[1].append(sample)
    
strata["population"] = results[0]
strata["sampled_users"] = results[1]

In [ ]:
strata.tail(n = 20)

The users' projects need to be in domain form, so get a table mapping site keys to site domains:
```
select 
site_global_key as site_key,
substring(reverse(site_domain), 2) as site_domain
from enwiki.sites;
```

In [ ]:
domains = pd.read_table("site_domains.tsv", index_col = 0)

def lookup_domain(site_key):
    return domains.loc[site_key, "site_domain"]

domains.head()

In [ ]:
for row in strata.itertuples(index = False):
    if row.sampled_users:
        text = ""
        for user in row.sampled_users:
            text += "* {{{{target | user = {} | site = {}}}}}\n".format(user[0], lookup_domain(user[1]))
        site = pwb.Site()
        page = pwb.Page(site, "Community Engagement Insights/MassMessages/Lists/2016/" + row.code)
        page.text = text
        page.save(u"Upload a target list for the 2016 Performance Survey", minor = False)